# 0. Importing Necessary Packages

In [1]:
# Printing the versions of packages
from importlib_metadata import version
for pkg in ['numpy', 'matplotlib', 'pandas', 'astropy', 'pyraf']:
    print(pkg+": ver "+version(pkg))

numpy: ver 1.20.3
matplotlib: ver 3.5.0
pandas: ver 1.3.5
astropy: ver 3.2.3
pyraf: ver 2.1.15


In [2]:
# matplotlib backend
%matplotlib notebook

# importing necessary modules
import time
import numpy as np
import glob, os
from matplotlib import pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import CartesianRepresentation
from astropy.coordinates import matching

current_dir = os.getcwd()
dir_iraf = os.path.abspath('/data2/iraf')    # where 'login.cl' file is located

os.chdir(dir_iraf)
from pyraf import iraf
os.chdir(current_dir)

# 1. Selecting PSF Stars in the Images

In [3]:
os.chdir('/data2/astronomicalobservation/')

In [4]:
dir_img = "n2041/"
imglist = [dir_img+"n2041b.fits", dir_img+"n2041v.fits"]
n_img = len(imglist)
band = ["b", "v"]
imglist

['n2041/n2041b.fits', 'n2041/n2041v.fits']

In [5]:
# You can also run this command in terminal.
ds9_options = "-scalemode zscale -scale lock yes -frame lock image "
names = ""
for i in np.arange(n_img):
    names += imglist[i]+" "
ds9_command = "ds9 "+ds9_options+names+"&"
print('Running "'+ds9_command+'" in the terminal...')
os.system(ds9_command)

Running "ds9 -scalemode zscale -scale lock yes -frame lock image n2041/n2041b.fits n2041/n2041v.fits &" in the terminal...


0

In [7]:
p0_x, p0_y = np.loadtxt(dir_img+"init_point.reg").T

In [8]:
iraf.noao()
iraf.digiphot()
iraf.ptools()

ptools/:
 istable        pexamine        tbcalc          tbsort          txselect
 pcalc          prenumber       tbconcat        txcalc          txsort
 pconcat        pselect         tbdump          txconcat
 pconvert       psort           tbrenumber      txdump
 pdump          pttest          tbselect        txrenumber


In [10]:
for i in np.arange(n_img):
    iraf.txdump("n2041/n2041"+band[i]+".mag", "ID,XCENTER,YCENTER,MAG,MSKY", "yes",
                Stdout="n2041/tmp")
    dat_phot = np.genfromtxt("n2041/tmp", dtype='float', encoding="ascii",
                             names=("ID","XCENTER","YCENTER","MAG","MSKY"))
    
    coord_regr = SkyCoord(p0_x*u.m, p0_y*u.m, 0.*u.m,
                          representation='cartesian') # python 3 representation_type
    coord_phot = SkyCoord(dat_phot['XCENTER']*u.m, dat_phot['YCENTER']*u.m, 0.*u.m,
                          representation='cartesian')
    idx, sep2d, dist3d = matching.match_coordinates_sky(coord_regr, coord_phot)
    
    f = open(dir_img+"n2041"+band[i]+".pst.0", "w")
    f.write("#N ID    XCENTER   YCENTER   MAG         MSKY \n")
    f.write("#U ##    pixels    pixels    magnitudes  counts \n")
    f.write("#F %-9d  %-10.3f   %-10.3f   %-12.3f     %-15.7g\n")
    f.write("#\n")
    for j in idx:
        f.write("{0:<9d} {1:<10.3f} {2:<10.3f} {3:<12.3f} {4:<15.7g}\n".format(int(dat_phot['ID'][j]),
                                                                     dat_phot['XCENTER'][j],
                                                                     dat_phot['YCENTER'][j],
                                                                     dat_phot['MAG'][j],
                                                                     dat_phot['MSKY'][j]))
    f.close()

In [11]:
iraf.noao()
iraf.digiphot()
iraf.daophot()

daophot/:
 addstar        daotest         nstar           pexamine        psf
 allstar        datapars@       pcalc           pfmerge         psort
 centerpars@    findpars@       pconcat         phot            pstselect
 daoedit        fitskypars@     pconvert        photpars@       seepsf
 daofind        group           pdump           prenumber       setimpars
 daopars@       grpselect       peak            pselect         substar


In [14]:
# img_g, hdr_g = fits.getdata(imglist[0], header=True)
# img_i, hdr_i = fits.getdata(imglist[1], header=True)

In [15]:
med_fwhm_g, med_fwhm_i = np.loadtxt(dir_img+"fwhms.txt").T
print(med_fwhm_g, med_fwhm_i)
fwhms = [med_fwhm_g, med_fwhm_i]

sky_value, sky_sigma = np.loadtxt(dir_img+"sky.txt")
print(sky_value, sky_sigma)

2.765 2.88
[25.09 44.45] [9.478 9.512]


# 2. Making a PSF Image

In [16]:
# PSF task (non-interative)
# If you want to do this interatively, please set "interactive" = True.
# (But the interactive PSF task could be a very annoying to you :>)

start_time = time.time()
imglist
for i in np.arange(n_img):
    print("\n----- Starting PSF task for "+imglist[i]+" -----\n")
    imgname = imglist[i].split(".fits")[0]
    magfile = imgname+".mag"
    kwargs = {"fwhmpsf":fwhms[i], "sigma":sky_sigma[i],
              "datamax":16300.0, "readnoi":15.0, "epadu":2.0, "itime":1.0,
              "psfrad":4.0*fwhms[i], "fitrad":1.0*fwhms[i]}
    
    n_iter = 1
    while True:
        psffile = imgname+".psf."+str(n_iter)+".fits"
        seepsffile = imgname+".spsf."+str(n_iter)+".fits"
        pstfile = imgname+".pst."+str(n_iter)
        psgfile = imgname+".psg."+str(n_iter)
        os.system("rm -rfv "+psffile+" "+seepsffile+" "+pstfile+" "+psgfile)
    
        iraf.psf(image=imglist[i], photfile=magfile, pstfile=imgname+".pst."+str(n_iter-1),
                 psfimage=psffile, opstfile=pstfile, groupfile=psgfile, 
                 function="auto", varorder=0, interactive=False, **kwargs)
        
        iraf.seepsf(psffile, seepsffile)
        ds9_command = "ds9 -scalemode zscale "+seepsffile+" &"
        os.system(ds9_command)

        response = input("Are you satisfied? (Y/N)")
        if (response == "Y"):
            break
        elif (response == "N"):
            alsfile = imgname+".als."+str(n_iter)
            rejfile = imgname+".arj."+str(n_iter)
            subimage = imgname+".sub."+str(n_iter)+".fits"
            os.system("rm -rfv "+alsfile+" "+rejfile+" "+subimage)
            iraf.allstar(image=imglist[i], photfile=psgfile, psfimage=psffile,
                         allstarfile=alsfile, rejfile=rejfile, subimage=subimage, **kwargs)
            
            os.system("rm -rfv "+subimage)
            iraf.substar(image=imglist[i], photfile=alsfile, exfile=pstfile, psfimage=psffile,
                         subimage=subimage, **kwargs)
            
            ds9_command = "ds9 -scalemode zscale -scale lock yes -frame lock image "+ \
                          imglist[i]+" "+subimage+" &"
            os.system(ds9_command)
            
            n_iter += 1
        else:
            raise TypeError
            break
    
end_time = time.time()
print("\n--- PSF tasks take {0:.4f} sec ---".format(end_time-start_time))


----- Starting PSF task for n2041/n2041b.fits -----


Analytic psf function(s) (auto): 	Analytic psf function(s): auto
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (11.06): 	New psf radius: 11.06 scale units 11.06 pixels
Fitting radius in scale units (2.765): 	New fitting radius: 2.765 scale units 2.765 pixels
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (16300.) (CR or value): 	New maximum good data value: 16300. counts


Computing PSF for image: n2041/n2041b.fits
792 stars read from n2041/n2041b.mag

Star 125 has been added to the PSF star list
	X:  198.89 Y:  144.19  Mag:  17.410  Dmin: 12.  Dmax: 775.
Star 108 has been added to the PSF star list
	X:  196.22 Y:  129.83  Mag:  17.160  Dmin: 12.  Dmax: 1034.
Star 110 has been added to the PSF star list
	X:  138.42 Y:  130.76  Mag:  17.012  Dmin: 13.  Dmax: 1102.
Star 150 has been added to the PSF star list
	X:  109.89 Y:  172.15  Ma

SUBTRACTING - Star:   50 X =  207.34 Y =   52.75 Mag =   19.75
REJECTING   - Star:   88 X =  251.99 Y =  106.64 Mag =   INDEF
SUBTRACTING - Star:   95 X =  136.03 Y =  114.04 Mag =   19.91
SUBTRACTING - Star:  107 X =  125.29 Y =  129.32 Mag =   20.42
REJECTING   - Star:  108 X =  196.22 Y =  129.83 Mag =   INDEF
REJECTING   - Star:  110 X =  138.39 Y =  130.76 Mag =   INDEF
SUBTRACTING - Star:  116 X =  209.27 Y =  135.25 Mag =   20.55
SUBTRACTING - Star:  120 X =  123.26 Y =  138.26 Mag =   20.23
REJECTING   - Star:  125 X =  198.89 Y =  144.19 Mag =   INDEF
SUBTRACTING - Star:  128 X =  188.56 Y =  145.58 Mag =   20.06
SUBTRACTING - Star:  130 X =  211.14 Y =  146.97 Mag =   19.93
SUBTRACTING - Star:  136 X =  201.00 Y =  157.71 Mag =   20.01
SUBTRACTING - Star:  141 X =   92.11 Y =  160.55 Mag =   19.65
SUBTRACTING - Star:  142 X =  105.36 Y =  161.72 Mag =   20.38
SUBTRACTING - Star:  145 X =   96.27 Y =  163.38 Mag =   20.65
REJECTING   - Star:  150 X =  109.89 Y =  172.14 Mag = 

Are you satisfied? (Y/N)N

Recenter the stars (yes): 	Recenter the stars: yes
Use group sky values (yes): 	Use group sky values: yes
Refit the sky (no): 	Refit the sky: no
Psf radius in scale units (11.52): 	New psf radius: 11.52 scale units 11.52 pixels
Fitting radius in scale units (2.88): 	New fitting radius: 2.88 scale units 2.88 pixels
Maximum group size in number of stars (60): 	New maximum group size: 60 stars
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (16300.) (CR or value): 	New maximum good data value: 16300. counts

NITER = 1
NITER = 2
NITER = 3
NITER = 4
FITTING:   ID:   116  XCEN:   209.11  YCEN:   135.68  MAG:    20.59
FITTING:   ID:   120  XCEN:   123.39  YCEN:   138.49  MAG:    20.44
NITER = 5
FITTING:   ID:    28  XCEN:   170.60  YCEN:    31.15  MAG:    20.25
FITTING:   ID:    35  XCEN:   203.10  YCEN:    33.26  MAG:    20.25
FITTING:   ID:    40  XCEN:   210.09  YCEN:    40.16  MAG:    18.44
FITTIN

SUBTRACTING - Star:  731 X =   85.20 Y =  433.11 Mag =   19.85
SUBTRACTING - Star:  732 X =   97.80 Y =  434.94 Mag =   20.40
REJECTING   - Star:  737 X =   95.11 Y =  442.46 Mag =   INDEF
SUBTRACTING - Star:  750 X =   91.40 Y =  458.45 Mag =   19.72

A total of 36 stars were subtracted out of a possible 50

Analytic psf function(s) (auto): 	Analytic psf function(s): auto
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (11.52): 	New psf radius: 11.52 scale units 11.52 pixels
Fitting radius in scale units (2.88): 	New fitting radius: 2.88 scale units 2.88 pixels
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (16300.) (CR or value): 	New maximum good data value: 16300. counts


Computing PSF for image: n2041/n2041v.fits
792 stars read from n2041/n2041v.mag

Star 125 has been added to the PSF star list
	X:  198.89 Y:  144.19  Mag:  16.602  Dmin: 27.  Dmax: 1528.
Star 108 has been added to th

# 3. Photometry Using the PSF Model Image

In [19]:
# ALLSTAR task
start_time = time.time()

for i in np.arange(n_img):
    print("\n----- Starting ALLSTAR task for "+imglist[i]+" -----\n")
    imgname = imglist[i].split(".fits")[0]
    magfile = imgname+".mag"
    psffile = sorted(glob.glob(imgname+".psf.*.fits"))[-1]
    
    kwargs = {"fwhmpsf":fwhms[i], "sigma":sky_sigma[i],
              "datamax":16300.0, "readnoi":15.0, "epadu":2.0, "itime":10.0,
              "psfrad":4.0*fwhms[i], "fitrad":1.0*fwhms[i]}

    alsfile = imgname+".als.A"
    rejfile = imgname+".arj.A"
    subimage = imgname+".sub.A.fits"
    os.system("rm -rfv "+alsfile+" "+rejfile+" "+subimage)
    
    iraf.allstar(image=imglist[i], photfile=magfile, psfimage=psffile,
                 allstarfile=alsfile, rejfile=rejfile, subimage=subimage, **kwargs)

    ds9_command = "ds9 -scalemode zscale -scale lock yes -frame lock image "+ \
                  imglist[i]+" "+subimage+" &"
    os.system(ds9_command)
    
end_time = time.time()
print("\n--- ALLSTAR tasks take {0:.4f} sec ---".format(end_time-start_time))


----- Starting ALLSTAR task for n2041/n2041b.fits -----


Recenter the stars (yes): 	Recenter the stars: yes
Use group sky values (yes): 	Use group sky values: yes
Refit the sky (no): 	Refit the sky: no
Psf radius in scale units (11.06): 	New psf radius: 11.06 scale units 11.06 pixels
Fitting radius in scale units (2.765): 	New fitting radius: 2.765 scale units 2.765 pixels
Maximum group size in number of stars (60): 	New maximum group size: 60 stars
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (16300.) (CR or value): 	New maximum good data value: 16300. counts

REJECTING: Star 320 has merged with star 324
NITER = 1
NITER = 2
NITER = 3
NITER = 4
FITTING:   ID:     3  XCEN:   255.38  YCEN:     6.75  MAG:    20.09
FITTING:   ID:     5  XCEN:    29.32  YCEN:     8.34  MAG:    19.88
FITTING:   ID:     7  XCEN:   280.72  YCEN:     8.87  MAG:    20.01
FITTING:   ID:     6  XCEN:    39.95  YCEN:     9.27  MAG:    20.94
FITT

FITTING:   ID:   392  XCEN:    37.86  YCEN:   284.15  MAG:    20.23
FITTING:   ID:   407  XCEN:   101.88  YCEN:   288.11  MAG:    16.79
FITTING:   ID:   405  XCEN:    42.73  YCEN:   288.10  MAG:    19.16
FITTING:   ID:   410  XCEN:   275.08  YCEN:   288.46  MAG:    19.57
FITTING:   ID:   426  XCEN:   184.89  YCEN:   293.06  MAG:    18.48
FITTING:   ID:   435  XCEN:   197.48  YCEN:   295.30  MAG:    19.42
FITTING:   ID:   436  XCEN:    70.18  YCEN:   295.99  MAG:    18.48
FITTING:   ID:   447  XCEN:   209.24  YCEN:   297.59  MAG:    17.91
FITTING:   ID:   449  XCEN:   279.38  YCEN:   298.25  MAG:    18.64
REJECTING: Star ID: 451 is too faint
FITTING:   ID:   452  XCEN:    33.03  YCEN:   300.40  MAG:    18.14
FITTING:   ID:   457  XCEN:    63.10  YCEN:   300.82  MAG:    19.64
FITTING:   ID:   460  XCEN:   199.13  YCEN:   301.70  MAG:    17.66
FITTING:   ID:   469  XCEN:   280.49  YCEN:   305.45  MAG:    19.07
FITTING:   ID:   476  XCEN:   223.12  YCEN:   308.37  MAG:    19.97
FITTING:   

FITTING:   ID:    58  XCEN:    67.90  YCEN:    66.13  MAG:    18.76
FITTING:   ID:    66  XCEN:   259.12  YCEN:    77.29  MAG:    19.86
FITTING:   ID:    67  XCEN:   105.34  YCEN:    77.99  MAG:    20.54
FITTING:   ID:    72  XCEN:   236.95  YCEN:    82.42  MAG:    20.32
FITTING:   ID:    74  XCEN:    62.36  YCEN:    84.26  MAG:    18.83
FITTING:   ID:    75  XCEN:   178.66  YCEN:    84.66  MAG:    19.76
FITTING:   ID:    78  XCEN:   274.39  YCEN:    87.97  MAG:    18.85
FITTING:   ID:    87  XCEN:   201.46  YCEN:   103.73  MAG:    20.00
FITTING:   ID:    88  XCEN:   251.99  YCEN:   106.64  MAG:    17.90
FITTING:   ID:    89  XCEN:   296.31  YCEN:   108.41  MAG:    19.40
FITTING:   ID:    90  XCEN:   194.76  YCEN:   109.90  MAG:    18.95
FITTING:   ID:    94  XCEN:   300.45  YCEN:   112.69  MAG:    16.99
FITTING:   ID:    99  XCEN:   115.35  YCEN:   121.02  MAG:    19.56
FITTING:   ID:   108  XCEN:   196.22  YCEN:   129.83  MAG:    17.14
FITTING:   ID:   106  XCEN:    81.78  YCEN:   13

FITTING:   ID:   412  XCEN:   189.72  YCEN:   289.24  MAG:    20.77
FITTING:   ID:   420  XCEN:   258.36  YCEN:   291.36  MAG:    18.18
FITTING:   ID:   424  XCEN:   267.68  YCEN:   291.81  MAG:    18.00
FITTING:   ID:   421  XCEN:    59.83  YCEN:   291.86  MAG:    20.36
FITTING:   ID:   425  XCEN:    75.21  YCEN:   293.00  MAG:    20.11
FITTING:   ID:   432  XCEN:   172.24  YCEN:   293.99  MAG:    19.28
FITTING:   ID:   427  XCEN:    17.83  YCEN:   294.63  MAG:    20.14
FITTING:   ID:   442  XCEN:   203.26  YCEN:   297.04  MAG:    18.74
FITTING:   ID:   446  XCEN:   191.05  YCEN:   298.32  MAG:    20.73
FITTING:   ID:   450  XCEN:    75.37  YCEN:   298.78  MAG:    20.05
FITTING:   ID:   454  XCEN:   125.61  YCEN:   300.01  MAG:    15.51
FITTING:   ID:   477  XCEN:   232.05  YCEN:   308.08  MAG:    18.86
FITTING:   ID:   478  XCEN:    22.36  YCEN:   308.94  MAG:    18.80
FITTING:   ID:   483  XCEN:   206.14  YCEN:   309.63  MAG:    20.17
FITTING:   ID:   489  XCEN:    10.15  YCEN:   31

FITTING:   ID:    40  XCEN:   209.79  YCEN:    39.84  MAG:    18.14
FITTING:   ID:    42  XCEN:    40.57  YCEN:    41.58  MAG:    17.92
FITTING:   ID:    59  XCEN:   154.82  YCEN:    65.73  MAG:    20.27
FITTING:   ID:    61  XCEN:   146.86  YCEN:    69.48  MAG:    20.07
FITTING:   ID:    64  XCEN:   220.55  YCEN:    74.21  MAG:    20.02
FITTING:   ID:    81  XCEN:   207.31  YCEN:    92.46  MAG:    18.91
FITTING:   ID:    97  XCEN:   224.91  YCEN:   119.47  MAG:    20.35
FITTING:   ID:   100  XCEN:   223.23  YCEN:   121.77  MAG:    20.63
FITTING:   ID:   139  XCEN:   162.31  YCEN:   159.02  MAG:    19.92
FITTING:   ID:   145  XCEN:    96.27  YCEN:   163.38  MAG:    20.65
FITTING:   ID:   157  XCEN:   171.85  YCEN:   179.17  MAG:    20.28
FITTING:   ID:   172  XCEN:   124.25  YCEN:   186.28  MAG:    19.71
FITTING:   ID:   168  XCEN:   221.11  YCEN:   184.46  MAG:    20.02
FITTING:   ID:   176  XCEN:   166.76  YCEN:   190.68  MAG:    19.57
FITTING:   ID:   189  XCEN:   195.47  YCEN:   20

...

FITTING:   ID:   160  XCEN:   187.33  YCEN:   182.15  MAG:    20.00
FITTING:   ID:   167  XCEN:   189.85  YCEN:   185.34  MAG:    19.12
FITTING:   ID:   162  XCEN:    92.77  YCEN:   182.94  MAG:    19.24
FITTING:   ID:   172  XCEN:   124.40  YCEN:   186.52  MAG:    19.93
FITTING:   ID:   171  XCEN:    42.95  YCEN:   187.67  MAG:    17.30
FITTING:   ID:   174  XCEN:    81.47  YCEN:   188.23  MAG:    18.66
FITTING:   ID:   175  XCEN:   227.59  YCEN:   189.90  MAG:    20.24
FITTING:   ID:   177  XCEN:   244.70  YCEN:   192.66  MAG:    20.13
FITTING:   ID:   179  XCEN:   128.58  YCEN:   196.96  MAG:    17.40
FITTING:   ID:   181  XCEN:   169.66  YCEN:   198.58  MAG:    18.98
FITTING:   ID:   182  XCEN:   112.45  YCEN:   200.71  MAG:    20.34
FITTING:   ID:   186  XCEN:   271.18  YCEN:   203.14  MAG:    20.82
FITTING:   ID:   190  XCEN:   257.51  YCEN:   205.74  MAG:    19.34
FITTING:   ID:   195  XCEN:   135.12  YCEN:   208.45  MAG:    18.27
FITTING:   ID:   194  XCEN:    99.81  YCEN:   20

FITTING:   ID:   617  XCEN:   199.73  YCEN:   356.22  MAG:    19.89
FITTING:   ID:   621  XCEN:   237.69  YCEN:   357.28  MAG:    19.01
FITTING:   ID:   626  XCEN:   143.09  YCEN:   360.92  MAG:    19.79
FITTING:   ID:   633  XCEN:   291.11  YCEN:   362.31  MAG:    19.93
FITTING:   ID:   649  XCEN:   110.05  YCEN:   370.03  MAG:    20.24
FITTING:   ID:   638  XCEN:   160.58  YCEN:   365.44  MAG:    18.54
FITTING:   ID:   640  XCEN:   196.30  YCEN:   366.62  MAG:    18.17
FITTING:   ID:   646  XCEN:   185.87  YCEN:   367.44  MAG:    19.46
FITTING:   ID:   645  XCEN:    80.94  YCEN:   367.47  MAG:    19.58
FITTING:   ID:   643  XCEN:    60.98  YCEN:   367.81  MAG:    17.63
FITTING:   ID:   641  XCEN:    22.49  YCEN:   368.15  MAG:    19.90
FITTING:   ID:   656  XCEN:   144.41  YCEN:   374.53  MAG:    19.13
FITTING:   ID:   655  XCEN:    21.43  YCEN:   374.73  MAG:    20.84
FITTING:   ID:   658  XCEN:   129.31  YCEN:   377.06  MAG:    20.48
FITTING:   ID:   660  XCEN:   116.56  YCEN:   37

FITTING:   ID:   467  XCEN:   139.76  YCEN:   305.58  MAG:    15.37
FITTING:   ID:   481  XCEN:   137.22  YCEN:   308.99  MAG:    16.97
FITTING:   ID:   466  XCEN:   105.58  YCEN:   305.79  MAG:    18.36
FITTING:   ID:   471  XCEN:   148.03  YCEN:   306.29  MAG:    14.60
FITTING:   ID:   477  XCEN:   232.16  YCEN:   308.77  MAG:    18.94
FITTING:   ID:   488  XCEN:   299.45  YCEN:   311.83  MAG:    18.58
FITTING:   ID:   519  XCEN:   199.18  YCEN:   319.63  MAG:    21.71
FITTING:   ID:   529  XCEN:   180.49  YCEN:   324.00  MAG:    19.53
FITTING:   ID:   532  XCEN:   125.27  YCEN:   325.48  MAG:    18.07
FITTING:   ID:   550  XCEN:   124.01  YCEN:   330.05  MAG:    16.90
FITTING:   ID:   525  XCEN:   100.79  YCEN:   323.50  MAG:    17.08
FITTING:   ID:   548  XCEN:   101.05  YCEN:   328.95  MAG:    19.13
FITTING:   ID:   542  XCEN:   152.59  YCEN:   328.15  MAG:    16.55
FITTING:   ID:   540  XCEN:   199.77  YCEN:   327.98  MAG:    18.82
FITTING:   ID:   547  XCEN:    28.47  YCEN:   32

FITTING:   ID:   628  XCEN:   173.86  YCEN:   360.25  MAG:    19.75
FITTING:   ID:   632  XCEN:   146.33  YCEN:   362.64  MAG:    19.04
FITTING:   ID:   636  XCEN:   110.01  YCEN:   365.37  MAG:    19.97
FITTING:   ID:   648  XCEN:   164.58  YCEN:   368.22  MAG:    20.44
FITTING:   ID:   651  XCEN:   298.10  YCEN:   371.47  MAG:    19.21
FITTING:   ID:   652  XCEN:   140.19  YCEN:   372.69  MAG:    19.19
FITTING:   ID:   659  XCEN:   141.21  YCEN:   377.47  MAG:    19.38
FITTING:   ID:   661  XCEN:   133.09  YCEN:   378.35  MAG:    20.93
FITTING:   ID:   680  XCEN:    67.65  YCEN:   388.57  MAG:    19.78
FITTING:   ID:   684  XCEN:   121.78  YCEN:   391.19  MAG:    20.49
FITTING:   ID:   686  XCEN:    90.10  YCEN:   393.37  MAG:    19.64
FITTING:   ID:   706  XCEN:   129.44  YCEN:   408.20  MAG:    17.88
FITTING:   ID:   736  XCEN:   214.35  YCEN:   440.08  MAG:    19.80
FITTING:   ID:   773  XCEN:   150.64  YCEN:   479.48  MAG:    20.63
NITER = 9
FITTING:   ID:    60  XCEN:   135.17  

FITTING:   ID:   439  XCEN:    95.89  YCEN:   297.44  MAG:    19.03
FITTING:   ID:   462  XCEN:   131.57  YCEN:   303.82  MAG:    15.28
FITTING:   ID:   499  XCEN:   123.12  YCEN:   315.35  MAG:    16.60
NITER = 15
FITTING:   ID:   402  XCEN:   176.79  YCEN:   286.54  MAG:    19.22
FITTING:   ID:   399  XCEN:   139.58  YCEN:   285.88  MAG:    17.69
FITTING:   ID:   434  XCEN:   138.96  YCEN:   295.66  MAG:    15.76
FITTING:   ID:   455  XCEN:   147.27  YCEN:   300.44  MAG:    15.86
FITTING:   ID:   516  XCEN:   153.63  YCEN:   320.42  MAG:    17.52
NITER = 16
FITTING:   ID:   346  XCEN:   100.73  YCEN:   267.16  MAG:    18.48
FITTING:   ID:   389  XCEN:   128.28  YCEN:   284.12  MAG:    17.20
NITER = 17
FITTING:   ID:   552  XCEN:    84.87  YCEN:   331.20  MAG:    19.99
NITER = 18
FITTING:   ID:   441  XCEN:   144.42  YCEN:   297.32  MAG:    15.41
FITTING:   ID:   509  XCEN:   121.70  YCEN:   318.69  MAG:    16.99
NITER = 19
FITTING:   ID:   242  XCEN:   157.18  YCEN:   234.59  MAG:   

# 4. Finding Stars Again (if you want)

In [33]:
# 2nd photometry for the subtracted image
# This process is for adding more stars than the previous results by re-running DAOFIND, PHOT, and ALLSTAR tasks.
start_time = time.time()
for i in np.arange(n_img):
    imgname = imglist[i].split(".fits")[0]
    subimage = imgname+".sub.A.fits"
    coofile = imgname+".coo.sub1"
    os.system("rm -rfv "+coofile)
    
    kwargs = {"fwhmpsf":fwhms[i], "sigma":sky_sigma[i], "threshold":2.5, "verify":False}
    iraf.daofind(image=subimage, output=coofile, **kwargs)

    magfile = imgname+".mag.sub1"
    os.system("rm -rfv "+magfile)
    
    kwargs = {"fwhmpsf":fwhms[i], "sigma":sky_sigma[i],
              "datamax":16300.0, "readnoise":15.0, "epadu":2.0, "itime":1.0,
              "cbox":1.5*fwhms[i], "annulus":4*fwhms[i], "dannulus":2*fwhms[i],
              "skyvalue":sky_value[i], "apertures":1.0*fwhms[i], "zmag":0, "verify":False}
    iraf.phot(image=subimage, coords=coofile, output=magfile, **kwargs)

    psffile = sorted(glob.glob(imgname+".psf.*.fits"))[-1]
    als0file = imgname+".als.A"    # Previous PSF photometric results
    alsfile = imgname+".als.sub1"    # PSF photometric results from the subtracted image
    rejfile = imgname+".arj.sub1"
    subimage2 = imgname+".sub.sub1.fits"
    os.system("rm -rfv "+alsfile+" "+rejfile+" "+subimage2)

    kwargs = {"fwhmpsf":fwhms[i], "sigma":sky_sigma[i],
              "datamax":16300.0, "readnoi":15.0, "epadu":2.0, "itime":1.0,
              "psfrad":4.0*fwhms[i], "fitrad":1.0*fwhms[i]}    
    iraf.allstar(image=subimage, photfile=magfile, psfimage=psffile,
                 allstarfile=alsfile, rejfile=rejfile, subimage=subimage2, **kwargs)
    
    alsnewfile = imgname+".als.new1"    # Merged file of *.als.A and *.als.sub1
    os.system("ls -1 "+als0file+" > merge.lis")
    os.system("ls -1 "+alsfile+" >> merge.lis")
    iraf.pfmerge("@merge.lis", alsnewfile)
    iraf.prenumber(alsnewfile)
    os.system("rm -rfv "+alsfile)
    
    alsfinal = imgname+".als.B"    # The final PSF photometry file
    rejfinal = imgname+".arj.B"
    subfinal = imgname+".sub.B.fits"
    iraf.allstar(image=imglist[i], photfile=alsnewfile, psfimage=psffile,
                 allstarfile=alsfinal, rejfile=rejfinal, subimage=subfinal, **kwargs)
    os.system("rm -rfv "+alsfile+" "+alsnewfile)
    iraf.txdump(alsfinal, "ID,XCENTER,YCENTER,MAG,MSKY", "yes", Stdout=alsfinal+".dat")

    ds9_command = "ds9 -scalemode zscale -scale lock yes -frame lock image "+ \
                  imglist[i]+" "+subfinal+" &"
    os.system(ds9_command)
    
end_time = time.time()
print("\n--- re-DAOFIND, PHOT, ALLSTAR tasks take {0:.4f} sec ---".format(end_time-start_time))

removed 'n2041/n2041b.coo.sub1'

Image: n2041/n2041b.sub.A.fits  fwhmpsf: 2.765  ratio: 1.  theta: 0.  nsigma: 1.5

  300.06     4.02   -0.007   0.402   0.228  -0.333     1
  221.27     6.74   -0.044   0.349  -0.386  -0.351     2
   53.91    11.21   -0.050   0.325  -0.284  -0.601     3
  211.03    11.00   -0.456   0.892   0.173  -0.148     4
  129.59    12.12   -0.010   0.526  -0.126   0.560     5
   97.46    13.79   -0.133   0.536   0.341  -0.029     6
  156.24    23.75   -0.060   0.534  -0.079  -0.315     7
  154.80    36.93   -0.047   0.509   0.463   0.143     8
  248.34    37.33   -0.267   0.597  -0.854  -0.085     9
   37.32    39.25   -0.580   0.481   0.738  -0.631    10
   37.36    39.30   -0.645   0.682  -0.604  -0.822    11
  207.11    42.67   -0.094   0.863  -0.163  -0.225    12
  170.92    45.83   -0.029   0.389   0.851  -0.501    13
   77.49    48.20   -0.095   0.647  -0.302  -0.353    14
  166.84    50.72   -0.729   0.420   0.901  -0.042    15
   11.14    53.28   -0.781   

  143.55   261.23   -0.563   0.476  -0.914  -0.614   143
  108.65   263.05   -0.730   0.409  -0.093  -0.331   144
  129.30   262.55   -0.694   0.763   0.371   0.382   145
   75.32   263.81   -0.126   0.895   0.412  -0.181   146
  124.66   266.18   -1.908   0.499   0.548  -0.621   147
  139.14   266.04   -1.776   0.506  -0.679  -0.867   148
  150.67   266.38   -2.129   0.475   0.311   0.174   149
  162.75   266.25   -2.517   0.488   0.219  -0.207   150
   86.94   267.72   -0.086   0.465  -0.081  -0.199   151
   93.35   266.80   -0.482   0.303  -0.954  -0.055   152
  100.20   266.59   -2.178   0.546  -0.719  -0.458   153
  118.67   267.39   -2.747   0.578   0.956   0.018   154
  146.30   267.34   -1.436   0.473   0.074   0.914   155
  133.92   267.75   -1.456   0.539  -0.433  -0.461   156
   16.80   268.83   -0.622   0.495  -0.784  -0.653   157
  158.65   270.17   -0.965   0.436  -0.046  -0.526   158
   87.07   269.17   -0.297   0.403   0.446  -0.267   159
  113.11   271.41   -0.622   0.

  134.14   321.10   -2.522   0.882  -0.679   0.273   287
   90.42   322.79   -0.122   0.506   0.340   0.882   288
  137.49   324.61   -2.683   0.545  -0.835  -0.742   289
   25.07   324.92   -0.112   0.755   0.485  -0.231   290
  111.60   324.37   -0.297   0.449   0.189  -0.301   291
  191.93   324.83   -1.339   0.424   0.239  -0.282   292
  103.00   325.35   -0.056   0.659  -0.269  -0.485   293
  142.98   327.24   -2.041   0.473  -0.125   0.767   294
  149.30   327.12   -2.739   0.503   0.202   0.909   295
  182.19   326.70   -0.529   0.534  -0.627   0.284   296
  289.42   326.44   -0.134   0.544   0.027  -0.104   297
  116.04   328.98   -0.404   0.527  -0.681  -0.675   298
   97.71   331.06   -0.094   0.513   0.120  -0.114   299
   17.11   331.30   -0.010   0.303   0.274   0.174   300
   85.68   330.94   -1.148   0.545  -0.529  -0.155   301
  110.01   330.61   -1.417   0.564   0.434   0.296   302
  148.99   332.10   -0.101   0.690  -0.316  -0.601   303
  159.55   332.33   -1.638   0.

n2041b.sub.A.fits    170.92    45.83  23.46501   -5.197  ok
n2041b.sub.A.fits     77.49    48.20  22.70939   -6.175  ok
n2041b.sub.A.fits    166.84    50.72  22.55667   -6.402  ok
n2041b.sub.A.fits     11.14    53.28   26.4575    INDEF  ok
n2041b.sub.A.fits    222.74    59.73  22.22417   -6.258  ok
n2041b.sub.A.fits     43.56    61.02  23.22524   -6.007  ok
n2041b.sub.A.fits    221.65    65.10  23.03576   -6.552  ok
n2041b.sub.A.fits    195.78    71.04  22.14279   -6.040  ok
n2041b.sub.A.fits    158.49    75.75  23.06055   -6.003  ok
n2041b.sub.A.fits    279.53    75.79  22.18194   -6.193  ok
n2041b.sub.A.fits    125.65    80.49  23.54408   -6.066  ok
n2041b.sub.A.fits     22.25    83.76  23.00653   -6.466  ok
n2041b.sub.A.fits     62.84    92.38  24.24909   -5.951  ok
n2041b.sub.A.fits    247.26    92.32  23.25331   -4.521  ok
n2041b.sub.A.fits    258.45    99.93  22.64374   -6.151  ok
n2041b.sub.A.fits    298.55   111.85  23.18151    INDEF  ok
n2041b.sub.A.fits    290.94   115.36  23

n2041b.sub.A.fits    162.74   266.25  53.80466   -9.294  ok
n2041b.sub.A.fits     86.94   267.72  35.18893   -7.475  ok
n2041b.sub.A.fits     93.35   266.80  44.72955   -6.757  ok
n2041b.sub.A.fits    100.21   266.59  59.31498   -8.163  ok
n2041b.sub.A.fits    118.67   267.39  63.09441   -8.437  ok
n2041b.sub.A.fits    146.30   267.34  80.14567   -8.321  ok
n2041b.sub.A.fits    133.92   267.75   72.8576   -8.332  ok
n2041b.sub.A.fits     16.80   268.83  30.34366   -5.982  ok
n2041b.sub.A.fits    158.65   270.17  65.24482   -8.185  ok
n2041b.sub.A.fits     87.07   269.17  34.83252   -7.608  ok
n2041b.sub.A.fits    113.11   271.41   66.6114   -8.311  ok
n2041b.sub.A.fits    139.70   269.31  78.49779   -8.447  ok
n2041b.sub.A.fits      9.93   270.89   30.1785   -5.535  ok
n2041b.sub.A.fits     34.49   270.84  30.28934   -6.226  ok
n2041b.sub.A.fits     92.94   273.35  47.51168   -9.150  ok
n2041b.sub.A.fits    102.26   271.03  66.55109   -6.587  ok
n2041b.sub.A.fits    147.04   271.61  99

...

n2041v.sub.A.fits    124.60   367.66  53.79428   -4.652  ok
n2041v.sub.A.fits    298.02   367.57  45.88739   -7.374  ok
n2041v.sub.A.fits     52.74   371.91  48.25658   -6.118  ok
n2041v.sub.A.fits    118.75   372.21  51.36157   -5.801  ok
n2041v.sub.A.fits     99.25   373.15   50.6375   -3.230  ok
n2041v.sub.A.fits    158.34   375.06  50.92055   -6.730  ok
n2041v.sub.A.fits    169.89   375.87  49.54313   -6.096  ok
n2041v.sub.A.fits    275.42   376.33  44.57499   -6.215  ok
n2041v.sub.A.fits     79.20   376.61  49.34166   -5.092  ok
n2041v.sub.A.fits    161.94   376.14    51.083   -6.579  ok
n2041v.sub.A.fits    154.33   379.87  50.63222   -6.887  ok
n2041v.sub.A.fits    158.70   380.97  51.60208    INDEF  ok
n2041v.sub.A.fits    277.61   385.36  45.30557   -5.094  ok
n2041v.sub.A.fits    127.11   383.89  49.15363   -5.592  ok
n2041v.sub.A.fits     74.30   385.25  47.91085   -6.189  ok
n2041v.sub.A.fits    272.60   385.21  44.82766   -6.717  ok
n2041v.sub.A.fits     11.10   392.04  49

REJECTING: Star ID: 9 has too few good pixels
REJECTING: Star ID: 10 has too few good pixels
REJECTING: Star ID: 11 has too few good pixels
REJECTING: Star ID: 12 has too few good pixels
REJECTING: Star ID: 14 has too few good pixels
REJECTING: Star ID: 13 has too few good pixels
REJECTING: Star ID: 15 has too few good pixels
REJECTING: Star ID: 16 has too few good pixels
REJECTING: Star ID: 18 has too few good pixels
REJECTING: Star ID: 20 has too few good pixels
REJECTING: Star ID: 17 has too few good pixels
REJECTING: Star ID: 19 has too few good pixels
REJECTING: Star ID: 21 has too few good pixels
REJECTING: Star ID: 22 has too few good pixels
REJECTING: Star ID: 25 has too few good pixels
REJECTING: Star ID: 23 has too few good pixels
REJECTING: Star ID: 24 has too few good pixels
REJECTING: Star ID: 26 has too few good pixels
REJECTING: Star ID: 28 has too few good pixels
REJECTING: Star ID: 27 has too few good pixels
REJECTING: Star ID: 29 has too few good pixels
REJECTING: Sta

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



FITTING:   ID:   274  XCEN:   189.85  YCEN:   185.35  MAG:    19.11
FITTING:   ID:   283  XCEN:   112.45  YCEN:   200.71  MAG:    20.34
FITTING:   ID:   113  XCEN:   157.28  YCEN:   205.10  MAG:    20.53
FITTING:   ID:   482  XCEN:   287.16  YCEN:   209.11  MAG:    17.84
FITTING:   ID:   292  XCEN:    92.12  YCEN:   219.64  MAG:    19.75
FITTING:   ID:   586  XCEN:    95.32  YCEN:   222.82  MAG:    20.35
FITTING:   ID:   483  XCEN:    25.37  YCEN:   224.06  MAG:    18.03
FITTING:   ID:   587  XCEN:   168.19  YCEN:   224.49  MAG:    17.33
FITTING:   ID:   589  XCEN:    64.33  YCEN:   228.90  MAG:    18.34
FITTING:   ID:   590  XCEN:     1.93  YCEN:   236.79  MAG:    18.82
FITTING:   ID:   591  XCEN:   141.03  YCEN:   237.89  MAG:    20.60
FITTING:   ID:   697  XCEN:   138.80  YCEN:   240.31  MAG:    20.63
FITTING:   ID:   487  XCEN:   118.78  YCEN:   240.48  MAG:    19.15
FITTING:   ID:   488  XCEN:   119.12  YCEN:   245.52  MAG:    19.55
FITTING:   ID:   655  XCEN:   180.26  YCEN:   24

FITTING:   ID:   713  XCEN:   174.60  YCEN:   325.68  MAG:    19.18
FITTING:   ID:   684  XCEN:   147.35  YCEN:   348.16  MAG:    19.01
FITTING:   ID:   381  XCEN:   100.66  YCEN:   350.69  MAG:    17.65
FITTING:   ID:   685  XCEN:     3.27  YCEN:   352.51  MAG:    19.93
FITTING:   ID:   398  XCEN:   160.57  YCEN:   365.44  MAG:    18.55
FITTING:   ID:   562  XCEN:    91.44  YCEN:   432.50  MAG:    20.56
FITTING:   ID:   647  XCEN:   214.35  YCEN:   440.08  MAG:    19.80
FITTING:   ID:   692  XCEN:   234.44  YCEN:   442.42  MAG:    18.65
FITTING:   ID:   733  XCEN:    91.40  YCEN:   458.46  MAG:    19.72
FITTING:   ID:   223  XCEN:    53.06  YCEN:   459.42  MAG:    20.24
FITTING:   ID:   571  XCEN:   221.31  YCEN:   476.85  MAG:    19.12
FITTING:   ID:   648  XCEN:   150.64  YCEN:   479.48  MAG:    20.63
FITTING:   ID:   693  XCEN:   101.75  YCEN:   506.05  MAG:    18.04
NITER = 8
FITTING:   ID:   771  XCEN:   169.51  YCEN:    48.74  MAG:    19.89
FITTING:   ID:   653  XCEN:   124.41  

In [74]:
if glob.glob(dir_img+"PSF_Phot") == []:
    os.system("mkdir "+dir_img+"PSF_Phot")
for i in np.arange(n_img):
    imgname = imglist[i].split(".fits")[0]
    alsfinal = imgname+".als.B"
    subfinal = imgname+".sub.B.fits"
    psffile = sorted(glob.glob(imgname+".psf.*.fits"))[-1]
    spsffile = sorted(glob.glob(imgname+".spsf.*.fits"))[-1]
    
    os.system("cp -rpv "+alsfinal+" "+subfinal+" "+psffile+" "+spsffile+" "+dir_img+"PSF_Phot/")

In [21]:
from astropy.io import ascii

In [136]:
tblv = ascii.read('/data2/astronomicalobservation/n2041/n2041v.als.B')

In [61]:
tblb = ascii.read('/data2/astronomicalobservation/n2041/n2041b.als.B')

In [81]:
obj_catalog = coord_b
sky_catalog = coord_v

In [83]:
from astropy.coordinates import match_coordinates_sky
closest_ids, closest_dists, closest_dist3d = match_coordinates_sky(obj_catalog,sky_catalog)

In [92]:
def CROSS_MATCH(obj_catalog,sky_catalog,max_distance_second=5):
    from astropy.coordinates import match_coordinates_sky
    # input = obj_catalog(SkyCoord), sky_catalog(SkyCoord), max_distance_second(match threshold)
    # output = matched_obj_idx, matched_cat_idx, no_matched_obj_idx
    
    closest_ids, closest_dists, closest_dist3d = match_coordinates_sky(obj_catalog,sky_catalog)
    max_distance = max_distance_second/60
    matched_object_idx = []
    matched_catalog_idx =[]
    no_matched_object_idx = []
    for i in range(len(closest_dists)):
        if closest_dists.value[i] < max_distance:
            matched_object_idx.append(i)
            matched_catalog_idx.append(closest_ids[i])
        else:
            no_matched_object_idx.append(i)
    return matched_object_idx, matched_catalog_idx,no_matched_object_idx

In [125]:
coord_b = SkyCoord(bx*u.m, by*u.m, 0.*u.m,
                      representation='cartesian') # python 3 representation_type
coord_v = SkyCoord(vx*u.m, vy*u.m, 0.*u.m,
                      representation='cartesian')

In [128]:
matched_b, matched_v, no = CROSS_MATCH(coord_b,coord_v,2)

In [145]:
coord_b

<SkyCoord (ICRS): (x, y, z) in m
    [(255.38 ,   6.751, 0.), ( 29.325,   8.337, 0.),
     (280.715,   8.871, 0.), ( 39.948,   9.266, 0.),
     (216.505,  10.067, 0.), (219.698,  21.057, 0.),
     (305.968,  21.606, 0.), (133.267,  21.904, 0.),
     (109.454,  22.88 , 0.), (181.258,  25.572, 0.),
     (279.422,  28.437, 0.), (170.554,  30.763, 0.),
     ( 25.368,  31.378, 0.), (150.634,  32.841, 0.),
     ( 79.923,  37.443, 0.), (122.15 ,  41.782, 0.),
     (286.066,  46.043, 0.), ( 10.396,  51.193, 0.),
     (220.769,  55.704, 0.), (288.955,  58.622, 0.),
     (111.346,  61.782, 0.), ( 37.966,  62.667, 0.),
     (103.44 ,  63.976, 0.), (173.978,  72.417, 0.),
     (114.05 ,  73.469, 0.), (237.568,  76.433, 0.),
     (132.517,  77.588, 0.), ( 90.274,  81.81 , 0.),
     ( 32.469,  83.101, 0.), ( 17.065,  84.256, 0.),
     ( 28.165,  87.172, 0.), (230.356,  90.747, 0.),
     (160.716,  94.098, 0.), (222.897,  98.232, 0.),
     ( 96.095,  99.533, 0.), (136.776, 103.397, 0.),
     (201.463

In [141]:
coord_b[0]

<SkyCoord (ICRS): (x, y, z) in m
    (255.38, 6.751, 0.)>

In [143]:
coord_v[1]

<SkyCoord (ICRS): (x, y, z) in m
    (280.893, 9.257, 0.)>